In [ ]:
import socket
import os

print("=== Network Sandbox Test ===")
print(f"Hostname: {socket.gethostname()}")
print()

# Test 1: DNS resolution (should fail with --network=none)
try:
    addr = socket.getaddrinfo("example.com", 80)
    print(f"FAIL: DNS resolved example.com -> {addr[0][4]}")
except Exception as e:
    print(f"PASS: DNS resolution blocked ({type(e).__name__}: {e})")

print()

# Test 2: TCP connection (should fail)
try:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(3)
    s.connect(("8.8.8.8", 53))
    s.close()
    print("FAIL: TCP connection to 8.8.8.8:53 succeeded")
except Exception as e:
    print(f"PASS: TCP connection blocked ({type(e).__name__}: {e})")

print()

# Test 3: List network interfaces via /sys/class/net
try:
    ifaces = os.listdir("/sys/class/net")
    print(f"Network interfaces in container: {ifaces}")
except Exception as e:
    print(f"Cannot list interfaces: {e}")

In [ ]:
import os

print("=== Filesystem Sandbox Test ===")
print(f"CWD: {os.getcwd()}")
print(f"Workspace contents: {os.listdir('/workspace')}")
print()

# Test 1: Write to workspace (should fail - mounted read-only)
try:
    with open("/workspace/sandbox_test.txt", "w") as f:
        f.write("should not work")
    print("FAIL: wrote to /workspace (should be read-only)")
    os.remove("/workspace/sandbox_test.txt")
except Exception as e:
    print(f"PASS: workspace is read-only ({type(e).__name__}: {e})")

print()

# Test 2: Container root filesystem is writable (expected - it is ephemeral/overlay)
try:
    with open("/tmp/test_sandbox.txt", "w") as f:
        f.write("test")
    os.remove("/tmp/test_sandbox.txt")
    print("OK: container /tmp is writable (ephemeral overlay, not host filesystem)")
except Exception as e:
    print(f"Container /tmp is read-only: {e}")

print()

# Test 3: Verify mount layout
print("Mount points (from /proc/mounts):")
with open("/proc/mounts") as f:
    for line in f:
        if any(k in line for k in ["workspace", "ipc", " / "]):
            # highlight ro vs rw
            mode = (
                "READ-ONLY"
                if ",ro," in line or line.split()[3].startswith("ro,")
                else "read-write"
            )
            parts = line.strip().split()
            print(f"  {parts[1]:12s} -> {mode}  ({parts[0][:40]})")

In [ ]:
# print(AGENT_WAS_HERE)  # type: ignore

In [ ]:
print("agent was here")

In [ ]:
import time

for i in range(10):
    print(f"tick {i + 1}/10")
    time.sleep(1)
print("done\!")